In [1]:
!sudo -H pip install --upgrade youtube-dl

     |████████████████████████████████| 1.9 MB 5.4 MB/s 


In [5]:
!pip install keyframe

ERROR: Could not find a version that satisfies the requirement keyframe (from versions: none)
ERROR: No matching distribution found for keyframe


In [2]:
!youtube-dl https://www.youtube.com/watch?v=CJm7FbpDFsE&t=20s

[youtube] CJm7FbpDFsE: Downloading webpage
[download] Destination: Pedestrian Safety-CJm7FbpDFsE.f135.mp4
[download] 100% of 56.00MiB in 18:05
[download] Destination: Pedestrian Safety-CJm7FbpDFsE.f140.m4a
[download] 100% of 8.36MiB in 02:42
[ffmpeg] Merging formats into "Pedestrian Safety-CJm7FbpDFsE.mp4"
Deleting original file Pedestrian Safety-CJm7FbpDFsE.f135.mp4 (pass -k to keep)
Deleting original file Pedestrian Safety-CJm7FbpDFsE.f140.m4a (pass -k to keep)


In [29]:
# %cd data
!ls

In [ ]:
import os
import cv2
import subprocess
import numpy as np
filename = '/content/ped.mp4'

def get_frame_types(video_fn):
    command = 'ffprobe -v error -show_entries frame=pict_type -of default=noprint_wrappers=1'.split()
    out = subprocess.check_output(command + [video_fn]).decode()
    frame_types = out.replace('pict_type=','').split()
    return zip(range(len(frame_types)), frame_types)

def save_i_keyframes(video_fn):
    frame_types = get_frame_types(video_fn)
    i_frames = [x[0] for x in frame_types if x[1]=='I']
    
    count=0
    if i_frames:
        basename = os.path.splitext(os.path.basename(video_fn))[0]
        cap = cv2.VideoCapture(video_fn)
        ret, prev = cap.read()
        
        grayImage = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
        (thresh, prev) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
        for frame_no in i_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
            
            
            ret, frame = cap.read()
            grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            (thresh, frameNew) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
            diff = cv2.absdiff(prev, frameNew)
            non_zero_count = np.count_nonzero(diff)
            #print(non_zero_count)
            if non_zero_count>1000:
                print(non_zero_count)
                c=str(count)
                while len(c)!=3: c="0"+c
                outname = basename+'_i_frame_'+str(c)+'.jpg'
                cv2.imwrite(outname, frame)
                count+=1
                print ('Saved: '+outname)
            prev=frameNew
        cap.release()
    else:
        print ('No I-frames in '+video_fn)
"""
video_path = "video.mkv"
p_frame_thresh = 900000 # You may need to adjust this threshold

cap = cv2.VideoCapture(video_path)
# Read the first frame.
ret, prev_frame = cap.read()
frame_no=0
while ret:
    ret, curr_frame = cap.read()

    if ret:
        diff = cv2.absdiff(curr_frame, prev_frame)
        non_zero_count = np.count_nonzero(diff)
        if non_zero_count > p_frame_thresh:
            
            outname = "zzz"+str(frame_no)+'.jpg'
            frame_no+=1
            
            cv2.imwrite(outname, curr_frame)
        prev_frame = curr_frame
"""
if __name__ == '__main__':
    save_i_keyframes(filename) 